In [ ]:
import numpy as np
import pygetm

%matplotlib widget
import matplotlib.pyplot

## Set up the domain (Cartesian box)

In [ ]:
Lx, Ly = 100e3, 100e3
nx, ny = 100, 52
x = np.linspace(-Lx / 2, Lx / 2, nx)
y = np.linspace(-Ly / 2, Ly / 2, ny)
domain = pygetm.domain.create_cartesian(x, y, H=10.0, lat=0.0)

Am = 10.0
sim = pygetm.Simulation(
    domain,
    runtype=pygetm.RunType.BAROTROPIC_2D,
    momentum=pygetm.momentum.Momentum(Am=Am),
)

## Dirac delta in x-direction (not physical!)

In [ ]:
u = sim.U.array(fill=0.0)
v = sim.V.array(fill=0.0)
icenter = nx // 2 - 1
u[:, icenter] = 1.0

fig, ax = matplotlib.pyplot.subplots()
(model_line,) = ax.plot(u.grid.x[0, :], u[0, :], label="numerical")
ax.grid()
ax.set_ylabel("distance (m)")
ax.set_ylabel("u velocity (m s-1)")

## Animate

In [ ]:
timestep = 600.0
ntime = 1000
tol = 1e-15
diffU = sim.U.array(fill=0.0)
diffV = sim.V.array(fill=0.0)

xa = sim.U.x.values[0, :] - sim.U.x.values[0, icenter]
(analytical_line,) = ax.plot(u.grid.x[0, :], u[0, :], label="analytical")
ax.legend()

for itime in range(ntime):
    pygetm._pygetm.momentum_diffusion(sim.D_T_half, sim.X.D, u, v, Am, diffU, diffV)
    u.all_values += diffU.all_values / u.grid.D.all_values * timestep
    v.all_values += diffV.all_values / v.grid.D.all_values * timestep
    model_line.set_ydata(u[0, :])
    va = 2 * (2 * Am * itime * timestep)
    analytical = (
        1.0
        / np.sqrt(2 * np.pi * va)
        * np.exp(-0.5 * xa**2 / va)
        * sim.U.dx.values[0, :]
    )
    analytical_line.set_ydata(analytical)
    fig.canvas.draw()
assert (np.abs(u.ma - u.ma[0, :]) < tol).all()
assert (np.abs(v.ma) < tol).all()

# Dirac delta in y-direction (not physical!)

In [ ]:
u = sim.U.array(fill=0.0)
v = sim.V.array(fill=0.0)
jcenter = ny // 2 - 1
v[jcenter, :] = 1.0

fig, ax = matplotlib.pyplot.subplots()
(model_line,) = ax.plot(v.grid.y[:, 0], v[:, 0], label="numerical")
ax.grid()
ax.set_ylabel("distance (m)")
ax.set_ylabel("v velocity (m s-1)")

## Animate

In [ ]:
timestep = 600.0
ntime = 1000
tol = 1e-15
diffU = sim.U.array(fill=0.0)
diffV = sim.V.array(fill=0.0)

ya = sim.V.y.values[:, 0] - sim.V.y.values[jcenter, 0]
(analytical_line,) = ax.plot(v.grid.y[:, 0], v[:, 0], label="analytical")
ax.legend()

for itime in range(ntime):
    pygetm._pygetm.momentum_diffusion(sim.D_T_half, sim.X.D, u, v, Am, diffU, diffV)
    u.all_values += diffU.all_values / u.grid.D.all_values * timestep
    v.all_values += diffV.all_values / v.grid.D.all_values * timestep
    model_line.set_ydata(v[:, 0])
    va = 2 * (2 * Am * itime * timestep)
    analytical = (
        1.0
        / np.sqrt(2 * np.pi * va)
        * np.exp(-0.5 * ya**2 / va)
        * sim.V.dy.values[:, 0]
    )
    analytical_line.set_ydata(analytical)
    fig.canvas.draw()

assert (np.abs(v.ma - v.ma[:, 0, np.newaxis]) < tol).all()
assert (np.abs(u.ma) < tol).all()